In [ ]:
import torch
import data
from helpers import Config

In [ ]:
SEED = 42
DATA = '/floyd/input/ptb/'
CUDA = False
LOG_INTERVAL = 100
CONFIG_NAME = 'text_generation_base'
device = torch.device("cuda" if CUDA else "cpu")
args = Config(CONFIG_NAME)
assert args.temperature >= 1e-3, "temperature has to be greater or equal 1e-3"

In [ ]:
corpus = data.Corpus(DATA)
ntokens = len(corpus.dictionary)

In [ ]:
with open(args.checkpoint, 'rb') as f:
    model = torch.load(f, map_location=device)
model.eval()

In [ ]:
hidden = model.init_hidden(1)
cue = torch.randint(ntokens, (1, 1), dtype=torch.long).to(device)

In [ ]:
with open(args.outf, 'w') as outf:
    with torch.no_grad():  # no tracking history
        for i in range(args.words):
            output, hidden = model(cue, hidden)
            word_weights = output.squeeze().div(args.temperature).exp().cpu()
            word_idx = torch.multinomial(word_weights, 1)[0]
            cue.fill_(word_idx)
            print(cue.shape)
            word = corpus.dictionary.idx2word[word_idx]
            outf.write(word + ('\n' if i % 20 == 19 else ' '))
            if i % LOG_INTERVAL == 0:
                print('| Generated {}/{} words'.format(i, args.words))